# Jobs and Sessions - Code Laboratory

**Section 4: Running Circuits on IBM Quantum Hardware** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `job.result()` | `job.result(timeout=None)` | `PrimitiveResult` | Get results (BLOCKS) |
| `job.status()` | `job.status()` | `JobStatus` | Check state (non-blocking) |
| `job.job_id()` | `job.job_id()` | `str` | Save for later retrieval |
| `Session()` | `Session(backend, max_time=None)` | Context manager | Iterative algorithms (VQE) |
| `Batch()` | `Batch(backend)` | Context manager | Parallel independent circuits |

---

In [ ]:
"""
Qiskit Code Laboratory - Jobs and Sessions
==========================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.providers import JobStatus

# IBM Quantum Runtime imports  
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeKyoto

# Local primitives for demonstration
from qiskit.primitives import StatevectorSampler, StatevectorEstimator

# =============================================================
# UTILITY FUNCTIONS FOR THIS NOTEBOOK
# =============================================================

def show_job_status(job, label=""):
    """Display job status information."""
    print(f"{label} Job Status:")
    print(f"  done(): {job.done()}")

def create_test_circuit():
    """Create a simple test circuit."""
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    return qc

print("✅ Environment ready - using StatevectorSampler for demonstrations")

---

## `job.result()` and `job.status()`

### Signatures
```python
job.result(timeout: int = None) -> PrimitiveResult  # BLOCKS until complete
job.status() -> JobStatus                           # Returns immediately
```

### JobStatus Values
| Status | Meaning | Final? |
|--------|---------|--------|
| `QUEUED` | Waiting in queue | No |
| `RUNNING` | Currently executing | No |
| `DONE` | Completed successfully | Yes |
| `ERROR` | Failed execution | Yes |
| `CANCELLED` | User cancelled | Yes |

### See Also
- [README: Job Lifecycle](./README.md#jobs-and-execution)

In [ ]:
# ============================================================
# job.result() and job.status() - BASIC USAGE
# ============================================================

qc = create_test_circuit()
sampler = StatevectorSampler()

job = sampler.run([qc], shots=1000)

# job.result() BLOCKS until complete
result = job.result()
counts = result[0].data.meas.get_counts()

print("Job completed:")
print(f"  Results: {counts}")
print(f"  job.done(): {job.done()}")

print("\n✅ job.result() BLOCKS, job.status() returns immediately")

In [ ]:
# ============================================================
# JobStatus Enum - ALL VALUES
# ============================================================

print("JobStatus Enum Values:")
print("=" * 50)
print(f"  INITIALIZING: {JobStatus.INITIALIZING}")
print(f"  QUEUED:       {JobStatus.QUEUED}")
print(f"  VALIDATING:   {JobStatus.VALIDATING}")
print(f"  RUNNING:      {JobStatus.RUNNING}")
print(f"  DONE:         {JobStatus.DONE} ← Final")
print(f"  ERROR:        {JobStatus.ERROR} ← Final")
print(f"  CANCELLED:    {JobStatus.CANCELLED} ← Final")

print("\n💡 Final states: DONE, ERROR, CANCELLED")

---

## `Session()` - Dedicated Backend Access

### Signature
```python
Session(
    backend: Backend,
    max_time: str | int = None  # "25m", "2h", or seconds
) -> ContextManager
```

### Parameters
| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `backend` | `Backend` | Yes | Target backend |
| `max_time` | `str\|int` | No | Maximum session duration |

### Usage Pattern
```python
with Session(backend=backend) as session:
    sampler = Sampler(mode=session)  # NOT backend=backend!
    for iteration in range(n):
        job = sampler.run([circuit])
```

### See Also
- [README: Sessions](./README.md#sessions)

In [ ]:
# ============================================================
# Session Pattern - VQE-like Iteration
# ============================================================
# Note: Sessions require real hardware, this demos the pattern

backend = FakeManilaV2()

print("Session Pattern (VQE-like iterations):")
print("=" * 50)

# Simulating what Session would do on real hardware
sampler = StatevectorSampler()

for iteration in range(3):
    # VQE would update theta based on previous result
    theta = np.pi / (iteration + 2)
    
    qc = QuantumCircuit(2, 2)
    qc.ry(theta, 0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    
    # On real hardware: job = sampler.run([qc]) within session
    job = sampler.run([qc], shots=500)
    counts = job.result()[0].data.meas.get_counts()
    
    print(f"Iteration {iteration+1}: theta={theta:.3f}, counts={counts}")

print("\n💡 Session keeps backend reserved between iterations")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: session= vs backend= Parameter
# ============================================================

print("⚠️ TRAP: Session parameter confusion!")
print("=" * 55)

print("""
# ❌ WRONG - Using backend= inside Session
with Session(backend=backend) as session:
    sampler = Sampler(backend=backend)  # Wrong!
    
# ✅ CORRECT - Using mode= (or session= in older versions)
with Session(backend=backend) as session:
    sampler = Sampler(mode=session)     # Correct!
""")

print("💡 Inside Session: use mode=session, NOT backend=backend!")

---

## `Batch()` - Parallel Independent Circuits

### Signature
```python
Batch(backend: Backend) -> ContextManager
```

### Usage Pattern
```python
with Batch(backend=backend) as batch:
    sampler = Sampler(mode=batch)
    job = sampler.run([qc1, qc2, qc3])  # All run in parallel
```

### Session vs Batch
| Use Case | Mode |
|----------|------|
| VQE/QAOA (iterative) | Session |
| Parameter sweep | Batch |
| Independent circuits | Batch |

### See Also
- [README: Batch Execution](./README.md#batch-execution)

In [ ]:
# ============================================================
# Batch Pattern - Independent Circuits
# ============================================================

print("Batch Pattern (independent circuits):")
print("=" * 50)

sampler = StatevectorSampler()

# Create multiple independent circuits
circuits = []
for n_qubits in [2, 3, 4]:
    qc = QuantumCircuit(n_qubits, n_qubits)
    qc.h(range(n_qubits))
    qc.measure_all(add_bits=False)
    circuits.append(qc)

# Run all at once (in Batch, these would be parallel)
job = sampler.run(circuits, shots=500)
results = job.result()

for i, result in enumerate(results):
    counts = result.data.meas.get_counts()
    print(f"Circuit {i+1} ({circuits[i].num_qubits}q): {len(counts)} unique outcomes")

print("\n💡 Batch runs independent circuits in parallel")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: result() BLOCKS, status() Does Not
# ============================================================

print("⚠️ TRAP: Blocking vs non-blocking!")
print("=" * 55)

qc = create_test_circuit()
sampler = StatevectorSampler()
job = sampler.run([qc], shots=100)

# Non-blocking: returns immediately
is_done = job.done()
print(f"job.done() returns immediately: {is_done}")

# BLOCKING: waits until complete
result = job.result()  # This waits!
print(f"job.result() blocks until done")

print("""
Key difference:
  status() → Returns immediately (non-blocking)
  result() → Waits for completion (BLOCKS)
  done()   → Returns immediately (non-blocking)
""")

print("💡 On real hardware, job.result() can wait minutes!")

In [ ]:
# ============================================================
# CHALLENGE 1: Job Polling Pattern
# ============================================================
# Task: Create a polling loop that checks job status
# Expected: Pattern for monitoring jobs on real hardware
# ============================================================

def poll_job(job, max_checks=10, delay=0.1):
    """
    Poll job until done or max_checks reached.
    
    Returns:
        True if job completed, False otherwise
    """
    import time
    
    for i in range(max_checks):
        if job.done():
            return True
        time.sleep(delay)
    return False

# Test the pattern
qc = create_test_circuit()
sampler = StatevectorSampler()
job = sampler.run([qc], shots=1000)

# Poll until done
completed = poll_job(job)
print(f"Challenge 1: Job Polling")
print("=" * 50)
print(f"Job completed: {completed}")

if completed:
    result = job.result()
    counts = result[0].data.meas.get_counts()
    print(f"Results: {counts}")

print("\n✅ Challenge 1 PASSED - Polling pattern works!")

In [ ]:
# ============================================================
# CHALLENGE 2: Session vs Batch Decision
# ============================================================
# Task: Choose the right execution mode for each scenario
# Expected: Demonstrate understanding of when to use each
# ============================================================

def choose_execution_mode(scenario):
    """
    Determine the best execution mode for a scenario.
    
    Args:
        scenario: Description of the use case
        
    Returns:
        str: 'Session', 'Batch', or 'Job'
    """
    scenario_lower = scenario.lower()
    
    if any(word in scenario_lower for word in ['vqe', 'qaoa', 'iterative', 'adaptive']):
        return 'Session'
    elif any(word in scenario_lower for word in ['sweep', 'independent', 'parallel', 'batch']):
        return 'Batch'
    else:
        return 'Job'

# Test scenarios
scenarios = [
    "VQE optimization with 50 iterations",
    "Parameter sweep over 100 angles",
    "Single circuit test run",
    "QAOA with adaptive depth",
    "Independent error benchmarking circuits"
]

print("Challenge 2: Execution Mode Selection")
print("=" * 50)
for scenario in scenarios:
    mode = choose_execution_mode(scenario)
    print(f"  {scenario[:40]:40} → {mode}")

# Verify key decisions
assert choose_execution_mode("VQE") == 'Session'
assert choose_execution_mode("parameter sweep") == 'Batch'
print("\n✅ Challenge 2 PASSED - Mode selection works!")